## Install missing packages and import all required libraries

In [1]:
!pip3 install imbalanced-learn openpyxl tabulate
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
import pandas as pd, numpy as np, time, matplotlib.pyplot as plt
from sklearn.svm import SVC
from imblearn.combine import SMOTETomek
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, ConfusionMatrixDisplay

## Read the datasets in the original space and in the PCA transformed space in Ur and U matrix format

In [2]:
PHL = pd.read_excel("PHL.xlsx")
Ur = pd.read_excel("Ur.xlsx")
U = pd.read_excel("U.xlsx")
print(PHL)
print(Ur)
print(U)

       sy_snum   sy_pnum  pl_orbper   pl_rade  pl_bmasse   st_teff    st_lum  \
0     2.660890 -0.680655  -0.014752  1.052172   3.264503 -0.497570  3.196379   
1    -0.297639 -0.680655  -0.014718  1.082953   2.431395 -0.886529  3.451361   
2    -0.297639 -0.680655  -0.014776  1.175297   0.653926 -0.445366  2.541880   
3    -0.297639  0.180341  -0.014499  1.129125   1.250996 -0.059349 -0.070660   
4     5.619418 -0.680655  -0.014669  1.267641   0.113952  0.243583  0.270225   
...        ...       ...        ...       ...        ...       ...       ...   
4965  2.660890  1.041338  -0.014808  1.344594  -0.081391  0.542669  0.853819   
4966  2.660890  1.041338  -0.014767  1.082953   2.295483  0.542669  0.853819   
4967  2.660890  1.041338  -0.014585  1.113734   1.628506  0.542669  0.853819   
4968 -0.297639 -0.680655  -0.014741  1.375375  -0.113144 -0.428455  2.592331   
4969 -0.297639 -0.680655  -0.014785  1.221469   0.296386 -0.469630  2.645509   

       st_logg  Habitable  
0    -4.753

## Build model functions to test the datasets

### SVM Classifier

In [3]:
def svc_model_builder(X_train, y_train, X_test):
  svc = SVC(random_state = 12)
  param_grid = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['sigmoid', 'rbf']}
  clf = GridSearchCV(svc, param_grid, cv = 10, scoring = 'recall', return_train_score = False)
  clf.fit(X_train, y_train)
  return clf.predict(X_test)

MLP Classifier

In [4]:
def mlpc_model_builder(X_train, y_train, X_test):
  n_features = len(X.columns)
  mlpc = MLPClassifier(hidden_layer_sizes = [i for i in range(n_features - 2, 1, -2)], early_stopping = True, random_state = 12, max_iter = 10000)
  param_grid = {'solver': ['lbfgs', 'adam', 'sgd']}
  clf = GridSearchCV(mlpc, param_grid, cv = 10, scoring = 'recall', return_train_score = False)
  clf.fit(X_train.values, y_train)
  return clf.predict(X_test.values)

### KNN

In [5]:
def knn_model_builder(X_train, y_train, X_test):
  knn = KNeighborsClassifier()
  param_grid = {'n_neighbors': range(1, 3)}
  clf = GridSearchCV(knn, param_grid, cv = 10, scoring = 'recall', return_train_score = False)
  clf.fit(X_train, y_train)
  return clf.predict(X_test)

### Logistic Regression

In [6]:
def lr_model_builder(X_train, y_train, X_test):
  logreg = LogisticRegression(random_state = 12)
  param_grid = {'penalty': ['l2'], 'C': [0.01, 0.1, 1, 10, 100], 'solver': ['lbfgs'], 'class_weight' : ['balanced'], 'max_iter' : [10000]}
  clf = GridSearchCV(logreg, param_grid, cv = 10, scoring = 'recall', return_train_score = False)
  clf.fit(X_train, y_train)
  return clf.predict(X_test)

## Run the model functions for different resampling methods run on the various datasets since we have an imbalanced distribution

In [10]:
Dataset_names = ['PHL - All Features', 'Ur - All PCs', 'U - All PCs', 'PHL - Selected Features', 'Ur - Top 6 PCs', 'U - Top 6 PCs']
Model_names = ['SVM', 'MLPClassifier', 'KNN', 'Logistic Regression']
Sampler_names = ['ClusterCentroids', 'SMOTE', 'ADASYN', 'SMOTETomek']
metrics_df = pd.DataFrame(columns = ['Dataset', 'Model', 'Sampler', 'F1 Score', 'Precision', 'Sensitivity/Recall(TPR)', 'Specificity(TNR)', 'AUC', 'Time(sec)'])
for i, dataset in enumerate([PHL, Ur, U, PHL[["sy_pnum", "pl_orbper", "pl_rade", "pl_bmasse", "st_teff", "st_lum", "Habitable"]], Ur[["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "Habitable"]], U[["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "Habitable"]]]):
  for j, model in enumerate([svc_model_builder, mlpc_model_builder, knn_model_builder, lr_model_builder]):
    for k, sampler in enumerate([ClusterCentroids(random_state = 12), SMOTE(random_state = 12), ADASYN(random_state = 12), SMOTETomek(random_state = 12)]):
      start_time = time.time()
      X = dataset.iloc[:, :-1]
      y = dataset.iloc[:, -1]
      X_resampled, y_resampled = sampler.fit_resample(X, y)
      X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.33, random_state = 12, stratify = y_resampled)
      y_pred = model(X_train, y_train, X_test)
      disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels = ["Not Habitable", "Habitable"])
      disp.ax_.set_title(Model_names[j] + " - " + Dataset_names[i] + " - " + Sampler_names[k])
      plt.savefig(Model_names[j] + " - " + Dataset_names[i] + " - " + Sampler_names[k] + ".png")
      plt.close()
      metrics_df.loc[len(metrics_df.index)] = [Dataset_names[i], Model_names[j], Sampler_names[k], f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), np.mean(recall_score(y_test, y_pred, pos_label = 1)), roc_auc_score(y_test, y_pred), str(time.time() - start_time)]

/Users/sauravsjk/miniconda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Print the sorted results and write the data to an excel file

In [13]:
metrics_df_f1 = metrics_df.sort_values(by = ["F1 Score", "Sensitivity/Recall(TPR)", "Precision", "Specificity(TNR)", "AUC"], ascending = False)
metrics_df_recall = metrics_df.sort_values(by = ["Sensitivity/Recall(TPR)", "F1 Score", "Precision", "Specificity(TNR)", "AUC"], ascending = False)
print ("\nPerformance Results (F1 Score sorted):\n")
print (metrics_df_f1.to_markdown(index = False))
print ("\nPerformance Results (Recall sorted):\n")
print (metrics_df_recall.to_markdown(index = False))


Performance Results (F1 Score sorted):

| Dataset                 | Model               | Sampler          |   F1 Score |   Precision |   Sensitivity/Recall(TPR) |   Specificity(TNR) |      AUC |   Time(sec) |
|:------------------------|:--------------------|:-----------------|-----------:|------------:|--------------------------:|-------------------:|---------:|------------:|
| U - All PCs             | KNN                 | SMOTETomek       |   0.984793 |    0.971206 |                  0.998766 |           0.998766 | 0.984577 |    0.303152 |
| U - All PCs             | KNN                 | SMOTE            |   0.981796 |    0.966547 |                  0.997534 |           0.997534 | 0.981494 |    0.2735   |
| U - All PCs             | KNN                 | ADASYN           |   0.981258 |    0.963205 |                  1        |           1        | 0.980888 |    0.27903  |
| PHL - All Features      | KNN                 | SMOTE            |   0.977348 |    0.957963 |              

In [14]:
metrics_df_f1.to_excel('Metrics - F1 Score.xlsx')
metrics_df_recall.to_excel('Metrics - Recall.xlsx')

## References

1. KNN - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
2. MLPClassifier - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
3. SVM - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
4. Logistic Regression - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
5. ADASYN - https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.ADASYN.html
6. SMOTE - https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
7. SMOTETomek - https://imbalanced-learn.org/dev/references/generated/imblearn.combine.SMOTETomek.html
8. ClusterCentroids - https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.ClusterCentroids.html
9. GridSearchCV - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
10. Train_Test_Split - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
11. Matplotlib - https://matplotlib.org/stable/api/index
12. SKLearn Metrics - https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics
13. KNN Example - https://www.geeksforgeeks.org/k-nearest-neighbor-algorithm-in-python/
14. Pandas DataFrame sorting - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
15. Tabulate - https://pypi.org/project/tabulate/